In [ ]:
!pip install requests

In [ ]:
import requests

In [ ]:
# 웹서버에 요청을 보낸다. 
url = "http://127.0.0.1:8080/corona?ServiceKey=aaa"

response = requests.get(url)


In [ ]:
print(response)

In [ ]:
print(response.text)

In [ ]:
url = "http://127.0.0.1:8080/corona"
_param = {
    "ServiceKey" : 'aaa'
}
res = requests.get(url, params=_param)


In [ ]:
print(res)
print(res.text)